In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import getitem
import os
from dotenv import load_dotenv


In [3]:
load_dotenv()

True

In [4]:
loader = PyPDFDirectoryLoader("us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents= text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, 

In [5]:
len(final_documents)

316

In [6]:
embed = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device":"cpu"},
    encode_kwargs={'normalize_embeddings': True}
)

In [7]:
import numpy as np

print(np.array(embed.embed_query(final_documents[0].page_content)))
print(np.array(embed.embed_query(final_documents[0].page_content)).shape)

[-8.30601454e-02 -1.45066977e-02 -2.10276134e-02  2.72683091e-02
  4.53646928e-02  5.28341234e-02 -2.53758561e-02  3.61304060e-02
 -9.08312201e-02 -2.77017821e-02  7.97397867e-02  6.42474517e-02
 -3.54004055e-02 -4.04245406e-02 -1.13771502e-02  4.45296019e-02
 -3.88548896e-03 -3.79057694e-03 -4.54509854e-02  2.67047267e-02
 -2.05681697e-02  2.87432037e-02 -2.41201371e-02 -3.69412489e-02
  1.92780886e-02  1.06194736e-02  3.21833231e-03  2.33253441e-03
 -4.29321565e-02 -1.64999172e-01  2.77011818e-03  2.68276725e-02
 -4.12894525e-02 -1.88446753e-02  1.58918854e-02  9.22327861e-03
 -2.00687572e-02  8.16561133e-02  3.89413014e-02  5.52223660e-02
 -3.69984247e-02  1.75318550e-02 -1.28967166e-02  2.80620705e-04
 -2.51580868e-02  4.59335325e-03 -2.39578784e-02 -5.76560525e-03
  6.02942426e-03 -3.61178257e-02  3.84415239e-02 -1.75470184e-03
  5.05656339e-02  6.02409020e-02  4.52067815e-02 -4.91434969e-02
  1.82054006e-02 -1.46668348e-02 -2.53130645e-02  3.18243578e-02
  5.15597872e-02 -9.32343

In [8]:
vector_store=FAISS.from_documents(final_documents[:120],embed)

In [9]:
query="What is health insurance coverage?"
relevant_documents=vector_store.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
retriever = vector_store.as_retriever()

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

hf_api = os.environ['HUGGINGFACEHUB_API_TOKEN']

In [12]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

hf=HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B",
    huggingfacehub_api_token=hf_api,
    temperature=0.1,
    max_new_tokens=500,
)

response = hf.invoke("What is health insurance coverage?")
print(response)


 Health insurance coverage is a type of insurance that covers the cost of medical and surgical expenses incurred by the insured. Health insurance can reimburse the insured for expenses incurred from illness or injury, or pay the care provider directly. It is often included in employer benefit packages as a means of enticing quality employees, with premiums partially paid by the employer but often also deducted from employee paychecks. The cost of health insurance premiums is deductible to the payer, and the benefits received are tax-free, with certain specific exceptions, such as medical expenses that are eligible to be deducted from aggregate gross income, and employer payments that can be written off as a business expense.
Health insurance coverage is a type of insurance that covers the cost of medical and surgical expenses incurred by the insured. Health insurance can reimburse the insured for expenses incurred from illness or injury, or pay the care provider directly. It is often i

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Use ONLY the context below to answer the question.

Context:
{context}

Question:
{input}

Give ONLY the final answer. Do NOT include:
- Explanation
- Context
- Thinking
- Extra sections
- Markdown headers
""")

In [14]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    RunnablePassthrough.assign(
        context=itemgetter("input") | retriever
    )
    | prompt
    | hf

)

In [15]:
query = """
Differences in the uninsured rates by state in 2022
"""

In [18]:
response = rag_chain.invoke({"input":query})
print(response)

Answer:

The uninsured rate fell in 27 states (mainly states that had expanded Medicaid eligibility), while only Maine had an increase of 0.8 percentage points. Only one state saw a decrease in public coverage (Rhode Island), while seven states experienced decreases in private coverage. As groups, states that expanded Medicaid eligibility saw an increase in public coverage, while states that did not expand Medicaid eligibility saw an increase in private coverage from 2021 to 2022, although expansion states had both higher private and public coverage rates than nonexpansion states to start with in both 2021 and 2022. Massachusetts had the lowest uninsured rate and Texas had the highest in 2022.37 In 2022, Utah had the highest private coverage and lowest public coverage rate, while New Mexico had the highest public coverage and the lowest private coverage rate, consistent with the differences in the uninsured rates by state in 2022.

Explanation:

The uninsured rate fell in 27 states (ma